In [ ]:
#------------------------------------------------------------------------------------------
# 원본파일에서 text 추출해서 파일로 저장
# => 추출할 원본 파일 경로 폴더를 지정하면 해당 폴더에서 파일을 읽어와서 
# text 추출후 파일로 저장함.
#------------------------------------------------------------------------------------------
import os
import time
import numpy as np
import sys
import json
from tqdm import tqdm 
import numpy as np
import pandas as pd
from utils import MyUtils

#-----------------------------------------------
# Mpower Synap 추가
from os import sys
sys.path.append('../../../MpowerAI')
from pympower.classes.mshaai import MShaAI

shaai = MShaAI()
#-----------------------------------------------
myutils = MyUtils(yam_file_path='../data/test_settings.yaml')

file_folder = "../../../data11/doc_3_new/"# text 추출할 원본파일 경로 지정
files:list = []
len_list = []

filepaths = myutils.getListOfFiles(file_folder) # 폴더에 파일 path 얻어옴.

for filePath in tqdm(filepaths):
    srcFilePath = filePath
    tgtFilePath = srcFilePath.replace('/doc_3_new/', '/doc_3_new_extra/') # test 폴더를 test-extra로 만듬
    
    # text 추출 함.
    shaai.extract(srcPath=srcFilePath, tgtPath=tgtFilePath)

In [ ]:
#------------------------------------------------------------------------------------------
# text추출원본파일명 rename 처리
# => 원본파일명을 유니크한 이름으로 만들기 위해 맨 앞에 순서대로 숫자(01_)를 붙여줌.
#------------------------------------------------------------------------------------------

import os
import re
from tqdm import tqdm 
gcounter:int = 0

# 원본파일명 앞에 숫자를 붙여서 rename 시킴 
def rename_files_with_numbers(directory):
    global gcounter  # 전역 변수를 사용하겠다고 선언
    print(f'*directory:{directory}')
    print(f'*gcount:{gcounter}')
    
    # create a list of file and sub directories 
    listOfFile = os.listdir(directory)
    allFiles = list()
    # Iterate over all the entries
    for filename in tqdm(listOfFile):
        # Create full path
        fullPath = os.path.join(directory, filename)

        if os.path.isdir(fullPath):
            rename_files_with_numbers(fullPath)
        else:
            if '.ipynb_checkpoints' not in fullPath:
                # 새 파일명 생성
                new_filename = f"{gcounter+1:02d}_{filename}"
                # 정규 표현식을 사용하여 숫자와 밑줄을 제거
                #new_filename = re.sub(r'^\d{3}_', '', filename)
                # 새 파일 경로 생성
                new_path = os.path.join(directory, new_filename)

                # 파일명 변경
                os.rename(fullPath, new_path)
                gcounter += 1
                allFiles.append(new_path)
                

# 예시 디렉토리 경로
directory_path = "../../data11/test-extra"

# 함수 호출
rename_files_with_numbers(directory_path)

In [6]:
#------------------------------------------------------------------------------------------
# 파일명, 파일내용이 입력되면 folder_path에 파일을 만드는 함수
# -in:folder_path: 파일을 만들어질 폴더
# -in:rfile_name_list : 파일명 목록
# -in:rfile_text_list : 파일내용 목록들
#------------------------------------------------------------------------------------------
def mpower_save_doc(folder_path:str, rfile_name_list:list, rfile_text_list:list):
    
    file_path_list:list = []
    
    assert folder_path, f"folder_path is empty!"
    assert len(rfile_name_list) > 0, f"rfile_name_list is empty"
    assert len(rfile_text_list) > 0, f"rfile_text_list is empty"
    
    try:
        # 폴더가 없으며 새로 생성.
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        # rfile_name으로 파일 생성하고 rfile_text 내용을 write 함
        for rfile_name, rfile_text in zip(rfile_name_list, rfile_text_list):
            file_path = folder_path + "/" + rfile_name

            # text 파일로 저장 => 기존 파일이 있으면 덮어씀.
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(rfile_text)
            
            file_path_list.append(file_path)
    
    except Exception as e:
            msg = f'*mpower_save_doc is Fail!!..(error: {e})'
            #myutils.log_message(f'[info][/extract] {msg}')

    response = {"file_path_list":file_path_list}
    return response

In [7]:
res = mpower_save_doc(folder_path="./tmp1", rfile_name_list = rfile_name_list, rfile_text_list = rfile_text_list)

print(res)

{'file_path_list': ['./tmp1/F_d6c12004a90e1e7ac30f4e6ed50ad6ba.SMF', './tmp1/F_0126417bfc20c3cc88e77fd46b0c25bd.SMF', './tmp1/F_25546cc485f12a9f33be938f4735895a.SMF']}
